In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
import math
import random

In [2]:
def green(text):
    return f"\033[1;32m{text}\033[0m"


def red(text):
    return f"\033[1;31m{text}\033[0m"


def yellow(text):
    return f"\033[1;33m{text}\033[0m"


def bold(text):
    return f"\033[1m{text}\033[0m"

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"Using device: {green(device)}")

Using device: mps


In [ ]:
from dataclasses import dataclass


@dataclass
class TransformerConfig:
    d_model: int = 512
    d_ff: int = 2048

    num_heads: int = 8

    dropout: float = 0.1

    vocab_size: int = 10000
    max_seq_len: int = 512

### Word Embeddings


In [ ]:
class WordEmbedding(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.embedding = nn.Embedding(config.vocab_size, config.d_model)

    def forward(self, x):
        return self.embedding(x)

### Position Encoding


In [ ]:
class PositionEncoding(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.dropout = nn.Dropout(config.dropout)

        # Create the positional encoding matrix
        pe = torch.zeros(config.max_seq_len, config.d_model)
        position = torch.arange(0, config.max_seq_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, config.d_model, 2).float()
            * (-math.log(10000.0) / config.d_model)
        )

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[: x.size(0), :]
        return self.dropout(x)

In [ ]:
class LayerNorm(nn.Module):
    def __init__(self, in_features, eps=1e-6):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(in_features))
        self.bias = nn.Parameter(torch.zeros(in_features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        std = x.std(dim=-1, keepdim=True)
        return self.weight * (x - mean) / (std + self.eps) + self.bias

In [ ]:
class FFN(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.linear1 = nn.Linear(config.d_model, config.d_ff)
        self.linear2 = nn.Linear(config.d_ff, config.d_model)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [ ]:
def scaled_dot_production_attention(
    query: torch.Tensor,
    key: torch.Tensor,
    value: torch.Tensor,
    mask: torch.Tensor | None = None,
):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
        scores = scores.masked_fill(mask == 0, float("-inf"))

    attn_weights = F.softmax(scores, dim=-1)
    output = torch.matmul(attn_weights, value)

    return output, attn_weights

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.num_heads = config.num_heads
        self.d_model = config.d_model
        self.d_k = config.d_model // config.num_heads

        self.query = nn.Linear(config.d_model, config.d_model)
        self.key = nn.Linear(config.d_model, config.d_model)
        self.value = nn.Linear(config.d_model, config.d_model)

        self.fc_out = nn.Linear(config.d_model, config.d_model)
        self.dropout = nn.Dropout(config.dropout)

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        query = (
            self.query(query)
            .view(batch_size, -1, self.num_heads, self.d_k)
            .transpose(1, 2)
        )
        key = (
            self.key(key).view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        )
        value = (
            self.value(value)
            .view(batch_size, -1, self.num_heads, self.d_k)
            .transpose(1, 2)
        )

        attn_output, attn_weights = scaled_dot_production_attention(
            query, key, value, mask
        )

        attn_output = (
            attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        )
        output = self.fc_out(attn_output)

        return output, attn_weights

In [ ]:
class CausalAttention(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        mask = (
            torch.tril(torch.ones(config.max_seq_len, config.max_seq_len))
            .unsqueeze(0)
            .unsqueeze(0)
        )

        self.register_buffer("mask", mask)

    def forward(self, query, key, value):
        batch_size = query.size(0)

        query = query.view(batch_size, -1, 1, self.mask.size(-1))
        key = key.view(batch_size, -1, 1, self.mask.size(-1))
        value = value.view(batch_size, -1, 1, self.mask.size(-1))

        attn_output, attn_weights = scaled_dot_production_attention(
            query, key, value, self.mask
        )

        attn_output = attn_output.view(batch_size, -1, self.mask.size(-1))

        return attn_output, attn_weights

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.attention = MultiHeadAttention(config)
        self.ffn = FFN(config)

        self.norm1 = LayerNorm(config.d_model)
        self.norm2 = LayerNorm(config.d_model)

        self.dropout1 = nn.Dropout(config.dropout)
        self.dropout2 = nn.Dropout(config.dropout)

    def forward(self, x):
        attn_output, _ = self.attention(x, x, x)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)

        ffn_output = self.ffn(x)
        x = x + self.dropout2(ffn_output)
        x = self.norm2(x)

        return x


class DecoderBlock(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.attention1 = CausalAttention(config)
        self.attention2 = MultiHeadAttention(config)
        self.ffn = FFN(config)

        self.norm1 = LayerNorm(config.d_model)
        self.norm2 = LayerNorm(config.d_model)
        self.norm3 = LayerNorm(config.d_model)

        self.dropout1 = nn.Dropout(config.dropout)
        self.dropout2 = nn.Dropout(config.dropout)
        self.dropout3 = nn.Dropout(config.dropout)

    def forward(self, x, encoder_output):
        attn_output, _ = self.attention1(x, x, x)
        x = x + self.dropout1(attn_output)
        x = self.norm1(x)

        attn_output, _ = self.attention2(x, encoder_output, encoder_output)
        x = x + self.dropout2(attn_output)
        x = self.norm2(x)

        ffn_output = self.ffn(x)
        x = x + self.dropout3(ffn_output)
        x = self.norm3(x)

        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.embedding = WordEmbedding(config)
        self.position_encoding = PositionEncoding(config)

        self.layers = nn.ModuleList(
            [EncoderBlock(config) for _ in range(config.num_heads)]
        )

    def forward(self, x):
        x = self.embedding(x)
        x = self.position_encoding(x)

        for layer in self.layers:
            x = layer(x)

        return x


class Decoder(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.embedding = WordEmbedding(config)
        self.position_encoding = PositionEncoding(config)

        self.layers = nn.ModuleList(
            [DecoderBlock(config) for _ in range(config.num_heads)]
        )

    def forward(self, x, encoder_output):
        x = self.embedding(x)
        x = self.position_encoding(x)

        for layer in self.layers:
            x = layer(x, encoder_output)

        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, config: TransformerConfig):
        super().__init__()

        self.encoder = Encoder(config)
        self.decoder = Decoder(config)

        self.fc_out = nn.Linear(config.d_model, config.vocab_size)

    def tie_weights(self):
        """Tie the weights of the embedding and output layers."""
        self.fc_out.weight = self.encoder.embedding.embedding.weight

    def forward(self, src, tgt):
        encoder_output = self.encoder(src)
        decoder_output = self.decoder(tgt, encoder_output)
        output = self.fc_out(decoder_output)

        return output